Denne fil bliver brugt til at analysere og kigge på det datasæt vi har valgt at lave vores endelige projekt i. 



In [5]:
import kagglehub
import os
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("dsabljic/movielens-32m")
print("Path to dataset files:", path)

# List subfolder 'ml-32m'
dataset_folder = os.path.join(path, "ml-32m")
print("Indhold i mappen ml-32m:")
print(os.listdir(dataset_folder))

# Find en csv-fil i mappen
csv_files = [f for f in os.listdir(dataset_folder) if f.endswith('.csv')]

# Læs første CSV-fil (bare som eksempel)
if csv_files:
    first_csv = os.path.join(dataset_folder, csv_files[0])
    df = pd.read_csv(first_csv)
    print("Viser de første rækker af filen:", csv_files[0])
    print(df.head())
else:
    print("Ingen CSV-filer fundet i mappen.")


Path to dataset files: C:\Users\eriks\.cache\kagglehub\datasets\dsabljic\movielens-32m\versions\1
Indhold i mappen ml-32m:
['checksums.txt', 'links.csv', 'movies.csv', 'ratings.csv', 'README.txt', 'tags.csv']
Viser de første rækker af filen: links.csv
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
